### Changelog 

- Separating **process_iplist()** output from VT calls to folders (todays_date)
- changed output of **process_json()** FROM "_parsed-combined" to todays_date_parsed-combined in EACH RESPECTIVE FOLDER 

### Flow

1. User edits and feeds ip.csv
2. **process_iplist()** reads each and calls VT api 
3. Responses are stored in **"downloaded_vtresponse"** and seperated into folders by **respective dates** (DDMMYYYY)
4. **process_json()** reads jsons in each folder and generates a compilation for that day in each folder

### Questions / Todo List

**Update** 

I have managed to get the scripts running from jupyter notebook but have the following questions

(a) Receiving (via a web interface) either an individual or list of domais / IP addresses --> currently it's fed via CSV, should i create a front-end for people to upload their files?

(b) Storing list of domains / IP address into a queue based list --> Is this the back-end of things? That is to say, this script runs in the back-end and whenever files come in from front-end it'll trigger the script?

(c) Carryout enrichment --> Where does this "processed in the previous X days" come from? From what i understand, should i create a check such that when new information comes in, it will look at previous histories when the IP/Domain was checked, and continue / stop accordingly? 

(d) storing responses in disk and extracting subset into DB --> is there a specific subset you'd like? DB-wise I would prefer to try NoSQL as i have no experience with it!

In [1]:
import base64
import hashlib
import json
import requests
import time
import csv
import datetime
import os
import pandas as pd
from dateutil import tz
import pytz
from pymongo import MongoClient


json_template_ip = {
    
    "ip_address": "",
    "whois_date": "",
    "last_analysis_date": "",
    "reputation": "",
    "last_analysis_stats": "",
    "total_votes": "",
    "as_owner": "",
    "country": "",
    "asn": "",
    "image":"",
    "processed_date":"",
    "target_geo_country":""  ## input from original excel
       
}

# API_KEY = '0d9fdb6e32d74b9d12e3d894309531838c3aabe8d66b049fd3a7976fbedf2c68'  #@param  {type: "string"}
API_KEY = '207349263f9c5edd176cc079fa8000a5ab912df7d9e91154842c08031658675d'  #@param  {type: "string"}



client = MongoClient('localhost',27017)
# db = client['d_ip_enrich']
db = client['filtered_sg_ip_list_day1']
    


def process_iplist(filename_to_process, columnIndex, x_days_ago):
    
    print("======= process_iplist() START =======")
    
    # TODO: Make generalised and incorporate timestamp in foldername
    now = datetime.datetime.now()
    dt_string = now.strftime("%d%m%Y")
    d = datetime.timedelta(days = x_days_ago)
    deducted_date = (now - d).strftime("%d%m%Y")
    
    with open(filename_to_process + ".csv", newline='') as inputfile:

        if not os.path.exists("downloaded_vtresponse"):
                os.makedirs("downloaded_vtresponse")

        with open(filename_to_process + "_tracker_" + dt_string + ".csv", 'w', newline='') as outputfile:
            
            ip_list = csv.reader(inputfile, delimiter=',')
            output_writer = csv.writer(outputfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            counter = 0

            for row in ip_list:
                
                if counter == 0:
                    output_writer.writerow(row + ["Time Run"] + ["Processed?"])
                    outputfile.flush()
                    counter += 1
                    continue

                print("Processing: #" + str(counter) + " - " + row[columnIndex] + " Country: " + row[1])
                
                # Check if IP was proceeded x_days_ago, if yes, will skip calling
                file_skip = to_skip(row[0], "downloaded_vtresponse", x_days_ago)
                
                 # make dir to store API Responses
                if not os.path.exists("downloaded_vtresponse/" + dt_string):
                    os.makedirs("downloaded_vtresponse/" + dt_string)

                #Get an IP address Report
                if file_skip == 0:
                    r = requests.get("https://www.virustotal.com/api/v3/ip_addresses/"+row[0], headers={"x-apikey":API_KEY})
                    
                    ## Check status 200 = ok, 204 = exceeded, 400 = bad request, 403 = forbidden
                    # if status != 200, will break all processing
                    if (r.status_code != 200):
                        print("Status Code: ",r.status_code, "please take a look" )
                        output_writer.writerow(row + [datetime.datetime.now()] + ["Not Processed due to status_code: " + str(r.status_code)]) 
                        outputfile.flush()
                        break

                    r = r.json()
                    # inputting target_geo_country from excel into JSON
                    r['data']['attributes']['target_geo_country'] = row[1]
                    
                    with open("downloaded_vtresponse/" + dt_string + "/" + row[columnIndex] + ".json", "w") as outfile:

#                         outfile.write(r.text)
                        json_obj = json.dumps(r)
                        outfile.write(json_obj)

                    output_writer.writerow(row + [datetime.datetime.now()] + ["Processed"]) 

                    outputfile.flush()
                    time.sleep(16)

                
                else:
                    
                    output_writer.writerow(row + [datetime.datetime.now()] + ["Not Processed due to DUPLICATE in past "+ str(x_days_ago) + " days"] ) 
                    outputfile.flush()
                    
                
                counter += 1
                
    print("======= process_iplist() END ======= \n\n")
#                 time.sleep(16)


# check if file exist in folder_to_process during x_days_ago, returns 0 or 1
def to_skip(filename, folder_to_process, x_days_ago):
    
    now = datetime.datetime.now()
    dt_string = now.strftime("%d%m%Y")
    d = datetime.timedelta(days = x_days_ago)
    deducted_date = (now - d).strftime("%d%m%Y")
    to_skip = 0

    folders = os.listdir("downloaded_vtresponse")
    folders = os.listdir(folder_to_process)
    

    for folder in folders:

        if to_skip == 1:
            break

        # target folders within X days range
        if folder >= deducted_date:
            files_array = os.listdir("downloaded_vtresponse/" + folder)
#                         print(files_array)

            for file in files_array:
                filename_filetype = file.rsplit('.',1)
    #             print(filename_filedate)

                ## if filename == target THEN SKIP + WRITE A NOTE
                if filename == filename_filetype[0]:
                    print(f"file has been processed on {folder} which is <{x_days_ago} days ago, will skip API call")
                    to_skip = 1
                    break
        
    
    return to_skip


    #     print(os.listdir("downloaded_vtresponse/"+ folder))    

def process_json_folder(folder_to_process,json_template):
    
    print("======= process_json_folder() START =======")
    
    # Get Date + Time to input later
    now = datetime.datetime.now(pytz.timezone("Singapore"))
    dt_string = now.strftime("%d%m%Y")

    # Usual Folder: downloaded_vtresponse
    combined_df = pd.DataFrame()
    
    
    for filename in os.listdir(folder_to_process):
        f = os.path.join(folder_to_process, filename)
    
#         print("f:", f)
        # check if it is a file
        if os.path.isfile(f) and f[-5:]==".json":

            print("\n **** Processing:",f, "****")
            # Opening JSON file
            f = open(f)
#             print(f)

            # returns JSON object as
            # a dictionary
            data = json.load(f)
#             print(data)

            # load JSON template
            json_template = {
                            "ip_address": "",
                            "whois_date": "",
                            "last_analysis_date": "",
                            "reputation": "",
                            "last_analysis_stats": "",
                            "total_votes": "",
                            "as_owner": "",
                            "country": "",
                            "asn": "",
                            "image":"",
                            "processed_date":"",
                            "target_geo_country":""  ## input from original excel
                            }
        
            new_row = json_template
#             new_row = json_template()

            print("json_template: ", json_template)
            print("Fresh new_row from json_template: ", new_row)

            # populate fields in JSON template
            for key in new_row:    
#                 print("current key", key)

                try:
                    current_value = data['data']['attributes'][key]
                    
                    # replace epoch with legible date format for whois_date and last_analysis_date
                    if key[-4:] == "date":
                        to_zone = tz.gettz('Singapore')
                        date_time = datetime.datetime.fromtimestamp( current_value )  
#                         current_value = date_time.replace(tzinfo=to_zone)
                        date_time.replace(tzinfo=to_zone)
                        current_value = date_time
                        
            
                    new_row[key] = current_value

                except Exception as e: 
               
                    if key == "processed_date":
                        new_row[key] = now
                        print("new_row[key]:", now)
                    
                    elif key == "ip_address":
                        new_row[key] = data['data']['id']
                    
                    else:
                        print(key,"not found with exception:",e)

            print("new_row:", new_row)
            db.ip.insert_one(new_row)

            df_result = pd.json_normalize(new_row)
            
            combined_df = pd.concat([combined_df, df_result], ignore_index=True, sort=False)
    
    
#     print(combined_df)
    now = datetime.datetime.now(pytz.timezone("Singapore"))
    dt_string = now.strftime("%d%m%Y")
    
    combined_df.to_csv(folder_to_process + '/' + dt_string + '_parsed-combined.csv')
    
    print("======= process_json_folder() END ======= \n\n")



# Process the list of IPs (CSVs ok but IP must be x column in the list)
# (filename.csv, column, x_days_ago)
# process_iplist("ip", 0,7)
# process_iplist("filtered_sg_ip_list_day1", 0,7)
# process_iplist("filtered_sg_ip_list_day2", 0,7)
# process_iplist("filtered_sg_ip_list", 0,7)



# Process the downloaded VT JSONs
# process_json_folder("downloaded_vtresponse_10Jan_combinedFull5k")
process_json_folder("downloaded_vtresponse/28022023",json_template_ip)


print("completed")
exit(0)





======= process_json_folder() START =======

 **** Processing: downloaded_vtresponse/28022023\101.100.211.131.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
last_analysis_date not found with exception: 'last_analysis_date'
image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '101.100.211.131', 'whois_date': datetime.datetime(2021, 3, 17, 10, 32, 33), 'last_analysis_date': '', 'reputation': 0, 'last_analysis_stats': {'harmless': 83, 'malicious': 0, 'suspicious': 0,

image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '101.33.26.253', 'whois_date': datetime.datetime(2023, 2, 14, 1, 15, 25), 'last_analysis_date': datetime.datetime(2023, 2, 27, 1, 15, 21), 'reputation': 0, 'last_analysis_stats': {'harmless': 73, 'malicious': 0, 'suspicious': 0, 'undetected': 15, 'timeout': 0}, 'total_votes': {'harmless': 0, 'malicious': 0}, 'as_owner': 'Tencent Building, Kejizhongyi Avenue', 'country': 'SG', 'asn': 132203, 'image': '', 'processed_date': datetime.datetime(2023, 3, 1, 11, 51, 37, 658733, tzinfo=<DstTzInfo 'Singapore' +08+8:00:00 STD>), 'target_geo_country': 'SG'}

 **** Processing: downloaded_vtresponse/28022023\101.33.27.26.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from


 **** Processing: downloaded_vtresponse/28022023\103.252.202.238.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '103.252.202.238', 'whois_date': datetime.datetime(2019, 10, 20, 5, 32, 29), 'last_analysis_date': datetime.datetime(2022, 9, 19, 15, 24, 38), 'reputation': 0, 'last_analysis_stats': {'harmless': 87, 'malicious': 0, 'suspicious': 0, 'undetected': 0, 'timeout': 0}, 'total_votes': {'harmless': 0, 'malic

new_row: {'ip_address': '104.69.163.61', 'whois_date': '', 'last_analysis_date': datetime.datetime(2023, 1, 11, 11, 39, 17), 'reputation': 0, 'last_analysis_stats': {'harmless': 87, 'malicious': 0, 'suspicious': 0, 'undetected': 0, 'timeout': 0}, 'total_votes': {'harmless': 0, 'malicious': 0}, 'as_owner': 'AKAMAI-AS', 'country': 'SG', 'asn': 16625, 'image': '', 'processed_date': datetime.datetime(2023, 3, 1, 11, 51, 37, 658733, tzinfo=<DstTzInfo 'Singapore' +08+8:00:00 STD>), 'target_geo_country': 'SG'}

 **** Processing: downloaded_vtresponse/28022023\104.69.32.93.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner'


 **** Processing: downloaded_vtresponse/28022023\104.89.118.22.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
whois_date not found with exception: 'whois_date'
image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '104.89.118.22', 'whois_date': '', 'last_analysis_date': datetime.datetime(2023, 2, 20, 21, 42, 18), 'reputation': 0, 'last_analysis_stats': {'harmless': 87, 'malicious': 0, 'suspicious': 0, 'undetected': 0, 'timeout': 0}, 'total_votes': {'harmless': 0, 

...


 **** Processing: downloaded_vtresponse/28022023\54.192.150.44.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '54.192.150.44', 'whois_date': datetime.datetime(2023, 2, 5, 20, 57, 11), 'last_analysis_date': datetime.datetime(2023, 2, 27, 22, 41, 8), 'reputation': 0, 'last_analysis_stats': {'harmless': 87, 'malicious': 0, 'suspicious': 0, 'undetected': 0, 'timeout': 0}, 'total_votes': {'harmless': 0, 'malicious':


 **** Processing: downloaded_vtresponse/28022023\58.182.130.183.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '58.182.130.183', 'whois_date': datetime.datetime(2021, 4, 8, 15, 50, 32), 'last_analysis_date': datetime.datetime(2022, 9, 10, 20, 19, 33), 'reputation': 0, 'last_analysis_stats': {'harmless': 87, 'malicious': 0, 'suspicious': 0, 'undetected': 0, 'timeout': 0}, 'total_votes': {'harmless': 0, 'maliciou


 **** Processing: downloaded_vtresponse/28022023\74.125.200.109.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '74.125.200.109', 'whois_date': datetime.datetime(2023, 2, 20, 3, 45, 30), 'last_analysis_date': datetime.datetime(2023, 2, 20, 3, 40, 4), 'reputation': 0, 'last_analysis_stats': {'harmless': 87, 'malicious': 0, 'suspicious': 0, 'undetected': 0, 'timeout': 0}, 'total_votes': {'harmless': 0, 'malicious'


 **** Processing: downloaded_vtresponse/28022023\89.187.162.142.json ****
json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
Fresh new_row from json_template:  {'ip_address': '', 'whois_date': '', 'last_analysis_date': '', 'reputation': '', 'last_analysis_stats': '', 'total_votes': '', 'as_owner': '', 'country': '', 'asn': '', 'image': '', 'processed_date': '', 'target_geo_country': ''}
image not found with exception: 'image'
new_row[key]: 2023-03-01 11:51:37.658733+08:00
new_row: {'ip_address': '89.187.162.142', 'whois_date': datetime.datetime(2021, 9, 5, 5, 5, 49), 'last_analysis_date': datetime.datetime(2023, 2, 27, 22, 8, 58), 'reputation': 0, 'last_analysis_stats': {'harmless': 79, 'malicious': 0, 'suspicious': 0, 'undetected': 9, 'timeout': 0}, 'total_votes': {'harmless': 0, 'malicious':

completed


In [9]:
json_template_general = {
    
    "DNS": "", 
    "Whois": "",
    "whois_date": "", ## CONVERT FROM EPOCH TO USER FRIENDLY DATE
    "last_analysis_date": "",
    "creation_date": "",
    "reputation": "",
    "registrar": "",
    "last_analysis_stats": "",  ## SEPERATE INTO 5 COLUMNS?
    "last_https_certificate": "",
    "categories": "",
    "total_votes": "",
    "as_owner": "",
    "country": "",
    "asn": "",
    "download_archived_page":"",
    "image":"",
    "processed_date":""     ## OWN FIELD TO CHECK X+7 DAYS
    
    
}

In [2]:
json_template_ip = {
    
    "ip_address": "",
    "whois_date": "",
    "last_analysis_date": "",
    "reputation": "",
    "last_analysis_stats": "",
    "total_votes": "",
    "as_owner": "",
    "country": "",
    "asn": "",
    "image":"",
    "processed_date":"",
    "target_geo_country":""  ## input from original excel
       
}

In [15]:
json_template_domain = {
    
    "DNS": "", 
    "Whois": "",
    "whois_date": "",
    "last_analysis_date": "",
    "creation_date": "",
    "reputation": "",
    "registrar": "",
    "last_analysis_stats": "",
    "last_https_certificate": "",
    "categories": "",
    "total_votes": "",
    "download_archived_page":"",
    "image":"",
    "processed_date":""
    
    
}

In [19]:
for key in json_template_ip:
    json_template_ip[key] = 1 
    
json_template_ip

{'whois_date': 1,
 'last_analysis_date': 1,
 'reputation': 1,
 'last_analysis_stats': 1,
 'total_votes': 1,
 'as_owner': 1,
 'country': 1,
 'asn': 1,
 'image': 1}

In [36]:
class ip_template:

    # class attribute
#     ip_address = ""
#     whois_date = ""
#     last_analysis_date = ""
#     reputation= ""
#     last_analysis_stats=""
#     total_votes= ""
#     as_owner=""
#     country= ""
#     asn=""
#     image=""
#     processed_date=""
#     target_geo_country="" 
    def __init__(self, roll_no, name, batch):
        self.roll_no = roll_no
        self.name = name
        self.batch = batch
    
new_row = ip_template('','','')
# new_row['sg']
# new_row.country="sg"
new_row_str =json.dumps(new_row.__dict__)
new_row_json = json.loads(new_row_json)
a
# new_row_json
# type(new_row_json)
# # create parrot1 object
# parrot1 = Parrot()
# parrot1.name = "Blu"
# parrot1.age = 10

# # create another object parrot2
# parrot2 = Parrot()
# parrot2.name = "Woo"
# parrot2.age = 15

# # access attributes
# print(f"{parrot1.name} is {parrot1.age} years old")
# print(f"{parrot2.name} is {parrot2.age} years old")

{'roll_no': '', 'name': '', 'batch': ''}

In [ ]:
json_template_ip = {
    
    "ip_address": "",
    "whois_date": "",
    "last_analysis_date": "",
    "reputation": "",
    "last_analysis_stats": "",
    "total_votes": "",
    "as_owner": "",
    "country": "",
    "asn": "",
    "image":"",
    "processed_date":"",
    "target_geo_country":""  ## input from original excel
       
}

In [1]:
import base64
import hashlib
import json
import requests
import time
import csv
import datetime
import os
import pandas as pd
from dateutil import tz
import pytz
from pymongo import MongoClient

API_KEY = '207349263f9c5edd176cc079fa8000a5ab912df7d9e91154842c08031658675d'  #@param  {type: "string"}
r = requests.get("https://www.virustotal.com/api/v3/ip_addresses/1.1.1.1", headers={"x-apikey":API_KEY}) 



In [14]:
# r.content

r.status_code
# site_response = str(r.content)
# print(site_response)

200

In [3]:
x_days_ago = 7

["Not Processed due to DUPLICATE in past "+ str(x_days_ago) + " days"]

['Not Processed due to DUPLICATE in past 7 days']